# Overview

This notebook assumes you have already collected and scored MOC tweets. It creates a dataset for use in R to analyze the patterns of polarization over time. You will do some parsing on an AWS server and some locally before ultimately making a CSV file that you can open and analyze in R.

LH Note: on my computer, Git stuff and data live in different places, so you'll see notes about moving files or changing directories. I haven't figured out a good way to keep files in both places or to mirror or sync or whatever. So, for now, paths are hard-coded or there's a note about where to find a file.

# Get Data

2016 election data is on an AWS server under ```/data/purpletag```. 

To login: 

```ssh -i ~/.ssh/carolgrrr.pem ubuntu@purpletag.casmlab.org```

The data is large (> 4GB), so best to run Juypter notebooks to parse on the server. Then CSV files can be used locally.

You can run a notebook on the server and use your local browser with these two commands:

* ```ssh -L 8080:localhost:8888 -i ~/.ssh/carolgrrr.pem ubuntu@purpletag.casmlab.org```
* ```nohup jupyter notebook --no-browser > log.txt 2>&1 &```

Then access ```http://localhost:8080``` in your browser.

## On server: Parsing from ```scores``` files to CSV

This section assumes you have already run purpletag's ```collect``` and ```score``` functions and gotten the Twitter data that you want in JSON format and parsed that data into score files.

In [38]:
# Based on https://stackoverflow.com/questions/26415906/read-multiple-txt-files-into-pandas-dataframe-with-filename-as-column-header
import pandas as pd
import os
import glob
import yaml

def build_file_list(directory, extension):
    '''
    args: 
        directory - full path to where the files are
            ex: /data/purpletag/scores
        extension - tells us which files to include in the list
            ex: *.l.moc.scores # using 1-day purpletag MOC scores
    '''
    
    # Step 1: get a list of all score files in target directory
    fileList = []
    os.chdir( directory )

    # Step 2: Build up list of files:
    for files in glob.glob(extension): 
        fileName, fileExtension = os.path.splitext(files)
        fileList.append(files) #filename with extension
        
    return fileList

def build_df(fileList, outfile, score_type):
    '''
    args:
        fileList - list of files to include, usually output from build_file_list
        outfile - full path to where to put the df
            ex: /data/purpletag/mocs_by_date.pkl
    '''
    # Step 3: Build up DataFrame:
    # Based on https://stackoverflow.com/questions/35717706/python-how-to-turn-a-dictionary-of-dataframes-into-one-big-dataframe-with-colum
    d = {} # dictionary to hold multiple dfs

    for filename in fileList:
        df1 = pd.read_csv(filename, header=None, sep=' ', index_col=0)
        if score_type == 'moc': # moc score files
            d[filename[:-13]] = df1
        else: # tag score files
            d[filename[:-9]] = df1

    df = pd.concat(d, axis=1)
    df.columns = df.columns.droplevel(-1) 

    df.to_pickle(outfile)

In [39]:
# set some variables that get used a few times below
#git_path = "/home/ubuntu/purpletag" # where is the GitHub repo that holds this notebook
git_path = "/Users/libbyh/Documents/git/casmlab/purpletag/"
git_data_path = git_path + "2016-election-study/data-files/"
pt_path = "/data/purpletag" # where are the scores and other purpletag-generated data

In [16]:
fileList = build_file_list('/data/purpletag/scores', '*.1.moc.scores')
build_df(fileList, '/data/purpletag/mocs_by_date_test.pkl', 'moc')

Move the file from the AWS server to local if you want to work locally. For example, to move the file ```mocs_by_date.pkl``` from the server to my local repo, I use:

```scp -i ~/.ssh/carolgrrr.pem ubuntu@purpletag.casmlab.org:/data/purpletag/mocs_by_date.pkl ~/Documents/git/casmlab/purpletag/files/```

## Locally: Prepping for stats

We now have a pickled dataframe of the form handleXdate. We need to keep data only from Labor Day to Election Day and get weekly averages.

In [40]:
import pandas as pd

df = pd.read_pickle(git_data_path + 'mocs_by_date.pkl')
df.head()

,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,...,2016-10-30,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08
austinscottga08,NaN,2.40933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
benniegthompson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.85585,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bettymccollum04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-90.838,-61.3435,-40.482,NaN,-33.4513,NaN,NaN,NaN
billpascrell,NaN,NaN,-1.168750,-0.916501,-0.972477,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-17.9542,NaN,NaN,NaN,NaN,NaN,NaN
boblatta,NaN,NaN,0.100723,NaN,NaN,1.70286,NaN,1.56897,NaN,-0.031978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
def weekly_avg(df):
    '''
    Given a df from build_df, keep just the weeks we are interested in.
    '''
    week1_dates = ['2016-09-06','2016-09-07','2016-09-08','2016-09-09','2016-09-10','2016-09-11','2016-09-12']
    week2_dates = ['2016-09-13','2016-09-14','2016-09-15','2016-09-16','2016-09-17','2016-09-18','2016-09-19']
    week3_dates = ['2016-09-20','2016-09-21','2016-09-22','2016-09-23','2016-09-24','2016-09-25','2016-09-26']
    week4_dates = ['2016-09-27','2016-09-28','2016-09-29','2016-09-30','2016-10-01','2016-10-02','2016-10-03']
    week5_dates = ['2016-10-04','2016-10-05','2016-10-06','2016-10-07','2016-10-08','2016-10-09','2016-10-10']
    week6_dates = ['2016-10-11','2016-10-12','2016-10-13','2016-10-14','2016-10-15','2016-10-16','2016-10-17']
    week7_dates = ['2016-10-18','2016-10-19','2016-10-20','2016-10-21','2016-10-22','2016-10-23','2016-10-24']
    week8_dates = ['2016-10-25','2016-10-26','2016-10-27','2016-10-28','2016-10-29','2016-10-30','2016-10-31']
    week9_dates = ['2016-11-01','2016-11-02','2016-11-03','2016-11-04','2016-11-05','2016-11-06','2016-11-07']

    df['week1'] = df[week1_dates].mean(axis=1)
    df['week2'] = df[week2_dates].mean(axis=1)
    df['week3'] = df[week3_dates].mean(axis=1)
    df['week4'] = df[week4_dates].mean(axis=1)
    df['week5'] = df[week5_dates].mean(axis=1)
    df['week6'] = df[week6_dates].mean(axis=1)
    df['week7'] = df[week7_dates].mean(axis=1)
    df['week8'] = df[week8_dates].mean(axis=1)
    df['week9'] = df[week9_dates].mean(axis=1)
    
    return df

df = weekly_avg(df)
df.head()

,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,...,2016-11-08,week1,week2,week3,week4,week5,week6,week7,week8,week9
austinscottga08,NaN,2.40933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.932035,NaN,1.064290,3.675700,6.042600,1.561610,NaN,NaN
benniegthompson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.85585,NaN,NaN,...,NaN,NaN,-1.211380,NaN,NaN,NaN,-1.823010,NaN,NaN,NaN
bettymccollum04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-56.669016,-119.884760,-67.172900,-59.540265,-27.995798,-33.955595,-4.879673,-33.080633,-56.5287
billpascrell,NaN,NaN,-1.168750,-0.916501,-0.972477,NaN,NaN,NaN,NaN,NaN,...,NaN,-2.871430,-2.623810,-2.353348,-2.337897,NaN,-1.026550,NaN,-1.333330,-17.9542
boblatta,NaN,NaN,0.100723,NaN,NaN,1.70286,NaN,1.56897,NaN,-0.031978,...,NaN,1.379970,9.673527,NaN,NaN,NaN,0.974138,NaN,NaN,NaN


In [53]:
weekly_df = df[['week1','week2','week3','week4','week5','week6','week7','week8','week9']]
weekly_df["handle"] = weekly_df.index.str.lower() # this line throws the SettingWithCopyWarning which I'm ignoring
weekly_df.head()

/Users/libbyh/anaconda/envs/purpletag/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,week1,week2,week3,week4,week5,week6,week7,week8,week9,handle
austinscottga08,NaN,4.932035,NaN,1.064290,3.675700,6.042600,1.561610,NaN,NaN,austinscottga08
benniegthompson,NaN,-1.211380,NaN,NaN,NaN,-1.823010,NaN,NaN,NaN,benniegthompson
bettymccollum04,-56.669016,-119.884760,-67.172900,-59.540265,-27.995798,-33.955595,-4.879673,-33.080633,-56.5287,bettymccollum04
billpascrell,-2.871430,-2.623810,-2.353348,-2.337897,NaN,-1.026550,NaN,-1.333330,-17.9542,billpascrell
boblatta,1.379970,9.673527,NaN,NaN,NaN,0.974138,NaN,NaN,NaN,boblatta


In [54]:
weekly_df.sort_values(by = 'week2', ascending = True).head()

,week1,week2,week3,week4,week5,week6,week7,week8,week9,handle
replawrence,-28.383236,-308.088,-18.402877,-32.248687,0.698665,-43.500575,NaN,NaN,-0.811634,replawrence
repdennyheck,NaN,-300.145,-10.527903,-17.045025,NaN,-18.147370,-16.686711,-40.557700,-62.144650,repdennyheck
repbobbyrush,NaN,-270.786,-30.679700,-1.161955,NaN,NaN,NaN,-8.652900,-1.898720,repbobbyrush
nitalowey,-1.298840,-215.463,-39.049700,-6.699170,NaN,-11.493604,-1.084320,-3.043239,-18.510867,nitalowey
repcleaver,-6.302088,-194.110,-8.202654,-32.020135,-0.119293,0.375556,-18.248317,-3.684110,-9.614692,repcleaver


## Merge with legislator data
Now we merge our #polar-tag data with data about the legislators (e.g., party, state).

In [65]:
# get the data from Govtrack
with open(git_data_path + 'legislators-social-media-nov16.yaml', 'r') as f:
    df_social = pd.io.json.json_normalize(yaml.load(f))

with open(git_data_path + 'legislators-current.yaml', 'r') as f:
    df_current = pd.io.json.json_normalize(yaml.load(f))

# merge everything into one data frame with one row per MOC
df_meta = pd.merge(df_current, df_social, on="id.govtrack")
df_meta["handle"] = df_meta["social.twitter"].str.lower()

print("metadata length: " + str(len(df_meta)))

metadata length: 470


In [71]:
print("weekly_df length: " + str(len(weekly_df)))

df_merged = pd.merge(df_meta, weekly_df, on="handle", how='outer', indicator=True)
                     
print("df_merged length: " + str(len(df_merged)))

#cols_to_keep = ['id.govtrack','social.twitter','name.official_full','bio.gender','terms','week1','week2','week3','week4','week5','week6','week7','week8','week9']

df_merged = df_merged[['_merge','id.govtrack','handle','name.official_full','bio.gender','terms','week1','week2','week3','week4','week5','week6','week7','week8','week9']]

weekly_df length: 511
df_merged length: 529


In [72]:
# see where the merge failed and probably fix those by hand
df_merge_failed = df_merged.loc[df_merged['_merge'] != "both"]
df_merge_failed.to_csv(git_data_path + 'merge_failed.csv', encoding='utf-8')

Not sure why we have only 444 matches, but it's better than 12.

In [31]:
# get bioguide ids
bg_path = git_data_path + 'bioguide_ids.tsv'
bg = pd.DataFrame.from_csv(bg_path, sep='\t', header=0)
bg["Name"] = bg["Name"].str.strip()
bg = bg.rename(index=str, columns={"Member ID": "bioguide"})

bg.head()

,Name,Party - State,bioguide
Member,,,
"Abdnor, James (Republican - South Dakota)","Abdnor, James",Republican - South Dakota),A000009
"Abercrombie, Neil (Democratic - Hawaii)","Abercrombie, Neil",Democratic - Hawaii),A000014
"Abourezk, James (Democratic - South Dakota)","Abourezk, James",Democratic - South Dakota),A000017
"Abraham, Ralph Lee (Republican - Louisiana)","Abraham, Ralph Lee",Republican - Louisiana),A000374
"Abraham, Spencer (Republican - Michigan)","Abraham, Spencer",Republican - Michigan),A000355


In [32]:
# get Ballotpedia House results data
bp_path = git_data_path + 'house_results.csv'
bp = pd.DataFrame.from_csv(bp_path, sep=",", header=0)
bp.head()

,inc_name,inc_party,inc_ran,inc_won,challenger,winner_votes,runnerup_votes,winner_party,runnerup_party,Clinton 2016,Trump 2016,Obama 2012,Romney 2012,Obama 2008,McCain 2008
district,,,,,,,,,,,,,,,
AK-AL,"Young, Don",R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.6,52.8,41.2,55.3,38.1,59.7
AL-01,"Byrne, Bradley",R,1.0,1.0,0.0,"208,083","7,810",R,write in,34.1,63.5,37.4,61.8,38.5,60.9
AL-02,"Roby, Martha",R,1.0,1.0,1.0,"134,886","112,089",R,D,33.0,64.9,36.4,62.9,35.0,64.5
AL-03,"Rogers, Mike",R,1.0,1.0,1.0,"192,164","94,549",R,D,32.3,65.3,36.8,62.3,36.6,62.6
AL-04,"Aderholt, Rob",R,1.0,1.0,0.0,"235,925","3,519",R,write in,17.4,80.4,24.0,74.8,25.5,73.3


In [35]:
# match bioguide id data to name data from Ballotpedia
hr = pd.merge(bg, bp, left_on="Name", right_on="inc_name")
len(hr)

253

In [165]:
df1 = pd.concat([df_merged.drop(['terms'], axis=1), df_merged['terms'].apply(pd.Series)], axis=1)
df2 = pd.concat([df1.drop([0], axis=1), df1[0].apply(pd.Series)], axis=1)

keep_df = df2[['id.govtrack','social.twitter','name.official_full','bio.gender','type','party','week1','week2','week3','week4','week5','week6','week7','week8','week9']]
keep_df

,id.govtrack,social.twitter,name.official_full,bio.gender,type,party,week1,week2,week3,week4,week5,week6,week7,week8,week9
0,400050,SenSherrodBrown,Sherrod Brown,M,rep,Democrat,-0.222430,-1.141480,-2.894056,-28.188720,-0.587860,1.175650,-19.819673,-2.136967,-0.367748
1,300018,SenatorCantwell,Maria Cantwell,F,rep,Democrat,-118.229375,-12.501053,-5.904561,-39.350840,-11.031340,-20.399800,-33.510324,-18.325139,-28.551134
2,400064,SenatorCardin,Benjamin L. Cardin,M,rep,Democrat,-25.636298,-8.955374,-10.335548,-28.631307,0.866007,-25.627037,-16.434565,-21.518943,-17.753120
3,300019,SenatorCarper,Thomas R. Carper,M,rep,Democrat,-155.209127,-0.681740,-8.735320,-12.539793,-14.515218,-0.307598,-6.753922,-1.830320,-6.501631
4,412246,SenBobCasey,"Robert P. Casey, Jr.",M,sen,Democrat,-36.255503,-6.993170,-1.272941,-28.430050,-0.082187,-5.102067,-16.612726,-12.302640,-77.053400
5,412248,SenBobCorker,Bob Corker,M,sen,Republican,1.884995,8.134470,3.616570,7.727762,NaN,1.819670,NaN,1.748260,NaN
6,300043,SenFeinstein,Dianne Feinstein,F,sen,Democrat,-63.976412,-759.904700,-38.750658,-55.663120,-7.274102,-6.091820,-29.935897,NaN,-161.321474
7,300052,SenOrrinHatch,Orrin G. Hatch,M,sen,Republican,78.846271,155.737157,147.056983,155.817800,20.409763,66.542193,16.915500,52.884700,2.214622
9,412243,McCaskillOffice,Claire McCaskill,F,sen,Democrat,-4.946375,-728.367104,-52.026723,-3.490048,-0.763953,-10.002037,-0.857143,-2.249950,-0.748092
10,400272,SenatorMenendez,Robert Menendez,M,rep,Democrat,-36.628585,-9.758632,-0.829845,-29.610837,-2.319673,-19.566032,-16.767273,-6.268110,-242.438693


In [170]:
# melt it so each row is a person x week
df_long = pd.melt(keep_df, id_vars=['id.govtrack','social.twitter','name.official_full','bio.gender','party','type'],
                value_vars=['week1','week2','week3','week4','week5','week6','week7','week8','week9'],
                var_name='week', value_name='avg_score')
df_long['week'] = df_long['week'].str[-1:]

df_long.rename(columns = {'type':'chamber', 'social.twitter': 'handle', 'name.official_full': 'name', 'bio.gender': 'gender'}, inplace = True)

df_long.head()

,id.govtrack,handle,name,gender,party,chamber,week,avg_score
0,400050,SenSherrodBrown,Sherrod Brown,M,Democrat,rep,1,-0.222430
1,300018,SenatorCantwell,Maria Cantwell,F,Democrat,rep,1,-118.229375
2,400064,SenatorCardin,Benjamin L. Cardin,M,Democrat,rep,1,-25.636298
3,300019,SenatorCarper,Thomas R. Carper,M,Democrat,rep,1,-155.209127
4,412246,SenBobCasey,"Robert P. Casey, Jr.",M,Democrat,sen,1,-36.255503


In [175]:
# make sure we have just two parties
df_long.party.unique()

array(['Democrat', 'Republican'], dtype=object)

In [176]:
# get an absolute value of the polar score
df_long['abs'] = df_long['avg_score'].abs()

In [177]:
df_long.to_csv('data-files/weekly_averages_long.csv')

# Now move to R for analysis

Run ```~/Documents/git/casmlab/purpletag/2016_election.R```

That R script sends its output to ```2016_election_results.txt```

In [180]:
results = open('data-files/2016_election_results.txt', 'r')
print(results.read())


> # for pretty regression tables
> # http://stackoverflow.com/questions/30195718/stargazer-save-to-file-dont-show-in-console
> mod_stargazer <- functi .... [TRUNCATED] 

> df <- read.csv('weekly_averages_long.csv', header = TRUE, sep = ",", quote = "\"",
+                dec = ".", fill = TRUE, comment.char = "")

> summary(df)
       X           id.govtrack                 handle                 name      gender  
 Min.   :   0.0   Min.   :300002   AustinScottGA08:   9   Adam B. Schiff:   9   F: 792  
 1st Qu.: 998.8   1st Qu.:400326   BennieGThompson:   9   Adam Kinzinger:   9   M:3204  
 Median :1997.5   Median :412292   BettyMcCollum04:   9   Adam Smith    :   9           
 Mean   :1997.5   Mean   :401868   BillPascrell   :   9   Adrian Smith  :   9           
 3rd Qu.:2996.2   3rd Qu.:412533   BobLatta       :   9   Al Franken    :   9           
 Max.   :3995.0   Max.   :412674   BradSherman    :   9   Al Green      :   9           
                                   (Other)    

Based on the outlier-excluded linear mixed-effects models, it makes sense to remove RepThompson. The pattern stays the same even with RepThompson in the set though: negative effect for republican and week, positive effect for their interaction. ```lmm5``` is the model-of-best-fit. 

## Changing the way we score hashtags

What if we score tags for the 63-day period and then score MOCS?

Run the following (on the server) to get new scores:

* purpletag parse -t 63 -d 200
* purpletag score
* purpletag score --counts --score-mocs

That first command took a week because the code starts with today and works backwards 200 days, one day at a time. Each day takes over an hour. See Issue #18 about options for changing this behavior.

With the new tag measures, can start the process over. Start at "On Server: Parsing scores to CSV" with a new file name.

# Getting Tag Data for Paper

We need to know more about the tags people were using to make sense of the regression results. So, let's get some tag data.'

In [17]:
score_files = build_file_list('/data/purpletag/scores', '*.1.scores')
build_df(score_files, '/data/purpletag/scores_by_date.pkl', 'tag')

In [31]:
import pandas as pd

df = pd.read_pickle('/data/purpletag/scores_by_date.pkl')
df.head()

,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,...,2016-10-30,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08
02byyou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0h03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.09155,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000blackgirlbooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000culverts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df = weekly_avg(df)
df.head()

,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,...,2016-11-08,week1,week2,week3,week4,week5,week6,week7,week8,week9
02byyou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0h03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.09155,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000blackgirlbooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000culverts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_tags_weeks = df[['week1','week2','week3','week4','week5','week6','week7','week8','week9']]
df_tags_weeks = df_tags_weeks.dropna(how='all')
df_tags_weeks.head()
len(df_tags_weeks.index) # number of hashtags in our df

8788

What was happening in week 2 that made Democrats so polarized that week?

In [48]:
week2 = df_tags_weeks.sort_values(by = 'week2', ascending = True)
week2.head(10)

,week1,week2,week3,week4,week5,week6,week7,week8,week9
closetheloophole,NaN,-59.205900,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gunvote,NaN,-37.421430,-1.047950,NaN,NaN,NaN,NaN,NaN,NaN
doyourjob,-38.184000,-35.275373,-19.993510,-9.581359,-8.854950,-1.294718,-0.857143,-1.294215,NaN
noflynobuy,-2.327613,-31.117480,-0.895503,-2.447173,-1.000000,-0.639344,-0.848739,-1.150440,-0.862385
zika,-10.867272,-17.640425,-13.098810,0.378861,0.409471,0.770685,-0.523072,-0.240739,NaN
gunviolence,-12.015845,-16.567943,-1.425833,-1.305606,-5.510243,-8.491866,-4.862071,-1.556957,-1.096452
vawa,NaN,-9.794230,-0.876106,NaN,-1.043135,-1.099100,NaN,-1.255100,NaN
vawa22,NaN,-6.920000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
citizenshipday,NaN,-6.874340,NaN,NaN,NaN,NaN,NaN,NaN,NaN
flint,-7.977703,-6.397713,-17.578173,-10.841666,NaN,-0.920000,-1.714290,NaN,NaN


In [70]:
week2 = df_tags_weeks.sort_values(by = 'week2', ascending = False)
week2.head(10)

,week1,week2,week3,week4,week5,week6,week7,week8,week9
gitmo,0.925860,10.028098,2.044487,1.461175,1.181820,NaN,NaN,0.869231,NaN
afbday,NaN,9.922860,1.287210,NaN,NaN,NaN,NaN,NaN,NaN
nationalpeanutday,NaN,7.803470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
betterway,11.940174,7.387886,7.833811,7.556914,7.357540,7.865916,8.017330,13.162771,14.429357
obamacare,4.670520,6.241857,2.698930,5.810810,5.181158,5.950158,4.944576,14.130460,11.389783
vaaccountability,NaN,6.179620,NaN,NaN,1.034780,NaN,NaN,NaN,1.336730
powmiarecognitionday,NaN,6.114260,1.141410,NaN,NaN,NaN,NaN,NaN,NaN
constitutionday2016,NaN,5.199355,0.199447,NaN,NaN,NaN,NaN,NaN,NaN
choiceact,0.852349,5.028185,1.275000,1.064290,NaN,NaN,2.224915,NaN,NaN
missamerica,0.967105,4.642820,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
week9_dems = df_tags_weeks.sort_values(by = 'week9', ascending = True)
week9_dems.head(10)

,week1,week2,week3,week4,week5,week6,week7,week8,week9
flashbackfriday,1.70470,-2.422760,-2.052170,-0.004867,-0.846154,-0.911504,NaN,2.250000,-33.451300
wagegap,-1.17323,-1.645199,-1.026090,NaN,-0.654545,NaN,-0.857143,NaN,-32.916000
strongeramerica,NaN,-1.345185,NaN,NaN,-1.000000,-5.721689,-0.949151,-28.000000,-32.744850
latinaequalpay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-19.206510
studentloans,NaN,-1.120145,NaN,-1.247520,NaN,-16.377675,NaN,NaN,-18.123395
equalpay,-1.00350,-1.375968,-3.175365,-1.053165,-1.414139,-0.969027,-0.933947,-1.540167,-10.697953
protectpell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.085778
getcovered,NaN,NaN,NaN,-1.076390,NaN,NaN,NaN,NaN,-6.854688
trabajadoras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.732820
parisagreement,-2.92800,NaN,-1.609540,NaN,-4.748297,1.007904,-0.944954,NaN,-6.132178


In [72]:
week9_reps = df_tags_weeks.sort_values(by = 'week9', ascending = False)
week9_reps.head(10)

,week1,week2,week3,week4,week5,week6,week7,week8,week9
betterway,11.940174,7.387886,7.833811,7.556914,7.357540,7.865916,8.017330,13.162771,14.429357
obamacare,4.670520,6.241857,2.698930,5.810810,5.181158,5.950158,4.944576,14.130460,11.389783
tbt,1.753045,0.727676,-0.916108,-0.367255,0.955896,-0.550580,1.770165,-0.313871,3.279880
obamacarehorror,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.142590
wikileaks,NaN,NaN,NaN,NaN,NaN,NaN,1.058250,0.869231,2.673470
curesnow,2.131257,1.179782,1.091992,1.875733,2.307863,1.086960,2.088168,1.375054,2.568982
mobilebanking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.319150
china,-0.976000,0.876149,0.974576,NaN,1.008405,1.041670,0.989362,1.500000,2.277230
northdakota,-1.173230,NaN,NaN,NaN,NaN,NaN,NaN,0.750000,2.277230
obamacarefail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.340804,2.277230


# Go to the JSON for examples

In [1]:
import json
from datetime import datetime

# print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

missing = ['repboustany', 'repcorrinebrown', 'repduckworth', 'repgwengraham', 'repjoeheck', 
           'repjoepitts', 'repkirkpatrick', 'repmattsalmon', 'repmickmulvaney', 'repmikepompeo', 
           'repmiketurner', 'repmurphyfl', 'reprobbishop', 'repsamfarr', 'reptoddyoung', 'senatorsessions', 
           'senatortester', 'sentoomey', 'tiberipress', 'yvetteclarke']

# search JSON for high score tweeters
with open('/data/purpletag/jsons/1478545228.json','r') as f:
    for line in f:
        data = json.loads(line)
#         if (data['user']['screen_name'].lower() == 'replawrence' 
#             or data['user']['screen_name'].lower() == 'repdennyheck'): # most polarized Dems during Week 2
        if (data['user']['screen_name'].lower in missing):
            print(data['user']['screen_name'])
            print(data['created_at'])
            print(data['text'])
                        
# print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')) # takes about 2 min to search the whole thing

FileNotFoundError: [Errno 2] No such file or directory: '/data/purpletag/jsons/1478545228.json'

In [69]:
matches = list()

# search JSON for high score tags
with open('/data/purpletag/jsons/1478545228.json','r') as f:
    for line in f:
        data = json.loads(line)
        tags = data['entities']['hashtags']
        for tag in tags:
            if tag['text'].lower() == 'gunvote':
                matches.append(data['id'])
                print(data['user']['screen_name'])
                print(data['created_at'])
                print(data['text'])

print(len(matches))

RepWilson
Wed Sep 14 17:30:19 +0000 2016
American people want Congress to #DoYourJob. End gun violence in our community. Give us a vote. #NoFlyNoBuy  #CloseTheLoophole #GunVote
RepWilson
Wed Sep 14 17:10:09 +0000 2016
Enough moments of silence.  We need moments of action.  @SpeakerRyan: give us a vote #NoFlyNoBuy  #CloseTheLoophole #GunVote
RepWilson
Wed Sep 14 16:58:07 +0000 2016
Retweet if you stand with @HouseGVP call for @SpeakerRyan to allow a vote to close background check loophole #GunVote
RepWilson
Wed Sep 14 16:53:06 +0000 2016
@PewResearch: 85% of people support closing gun show loophole. @SpeakerRyan give us a vote #CloseTheLoophole #GunVote
RepWilson
Wed Sep 14 16:45:07 +0000 2016
Thousands killed w/ guns over reckless Republican recess. When will @SpeakerRyan give us a vote? #NoFlyNoBuy #CloseTheLoophole #GunVote
RepWilson
Wed Sep 14 16:32:05 +0000 2016
272 mass shootings this year. When will @SpeakerRyan give us a vote? #NoFlyNoBuy  #CloseTheLoophole #GunVote
NydiaVelazqu

In [ ]:
# check for missing data from alternate_route.ipynb
# search JSON for high score tags
with open('/data/purpletag/jsons/1478545228.json','r') as f:
    for line in f:
        data = json.loads(line)
        tags = data['entities']['hashtags']
        for tag in tags:
            if tag['text'].lower() == 'gunvote':
                matches.append(data['id'])
                print(data['user']['screen_name'])
                print(data['created_at'])
                print(data['text'])

print(len(matches))